<a href="https://colab.research.google.com/github/Muhammad1sys/PIAIC-Q2/blob/main/RAG_class4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai langchain-pinecone

In [44]:
!pip install -qU pypdf

In [45]:
import os
from google.colab import userdata
userdata.get('gemini-1.5-flash')
os.environ ['GOOGLE_API_KEY'] = userdata.get('gemini-1.5-flash')

In [46]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [47]:
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash")

In [48]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [49]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [50]:

embeddings.embed_query("What's our Q1 revenue?")

[0.01612485572695732,
 -0.03933568298816681,
 -0.002993757603690028,
 -0.033227454870939255,
 0.05022253096103668,
 0.03145093098282814,
 -0.00027826917357742786,
 -0.064089834690094,
 0.05403633043169975,
 0.02948707528412342,
 0.03588718920946121,
 0.024271711707115173,
 -0.035950273275375366,
 0.006472693756222725,
 0.029704688116908073,
 -0.05441908538341522,
 -0.03014720045030117,
 -0.013652290217578411,
 0.008419765159487724,
 -0.00975697860121727,
 -0.016383253037929535,
 0.061616990715265274,
 -0.028840864077210426,
 0.0015244809910655022,
 0.02118818275630474,
 0.02309279516339302,
 -0.009022949263453484,
 -0.022830545902252197,
 -0.03407009690999985,
 0.058693066239356995,
 -0.10280811786651611,
 0.06019517406821251,
 -0.03679637238383293,
 0.036128751933574677,
 0.0333784781396389,
 -0.05355016142129898,
 -0.02582419477403164,
 0.0417415127158165,
 -0.02746596373617649,
 0.025292860344052315,
 -0.02460378408432007,
 0.006439381279051304,
 -0.03763241693377495,
 -0.0278397221

# New Section

In [51]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

index_name = "quickstart4"
API_KEY = userdata.get('Pinecone_Api_Key')
pc = Pinecone(api_key= API_KEY)

# Check if index already exists before creating it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists.")

index = pc.Index(index_name)
vector_store = PineconeVectorStore(embedding=embeddings, index=index)



Index 'quickstart4' already exists.


In [52]:
from langchain.document_loaders import GoogleSpeechToTextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Import TextLoader from langchain.document_loaders
from langchain.document_loaders import TextLoader

#load documents
loader = TextLoader('document.txt') # replace with your file
documents = loader.load()

#split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)

[Document(metadata={'source': 'document.txt'}, page_content='Here are **50 multiple-choice questions (MCQs)** with correct answers based on the topics covered in the explanation provided. These questions are designed to meet industry standards and reflect the kinds of questions asked in interviews or technical assessments.\n\n---\n\n### **1. Terms of Service**\n1. What does a Terms of Service agreement typically include?  \n   - A. Guidelines for acceptable use  \n   - B. Data collection and sharing policies  \n   - C. Dispute resolution procedures  \n   - **D. All of the above**  \n\n2. What is the nature of a Terms of Service agreement?  \n   - A. An informal guideline  \n   - **B. A legally binding contract**  \n   - C. A user agreement that can be ignored  \n   - D. A suggestion for using the service'), Document(metadata={'source': 'document.txt'}, page_content='3. If a Terms of Service agreement includes "royalty-free" rights, what does it mean?  \n   - A. You lose ownership of yo

In [53]:
!pip install tqdm

In [56]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
vectors = []
doc_id_counter = 1  # Initialize a counter for document IDs

for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    # Generate a unique ID for each document using the counter
    doc_id = f"doc_{doc_id_counter}"
    doc_id_counter += 1  # Increment the counter for the next document
    # Upsert the vector with a unique ID and metadata
    index.upsert(vectors=[(doc_id, vector, {"text": doc.page_content})])  # include metadata

# After upserting, update the retriever to include the metadata key:
from langchain.vectorstores import Pinecone

# Assuming 'text' is the key for your metadata content
retriever = Pinecone(index=index, embedding=embeddings, text_key="text").as_retriever()

# Convert it into a retriever compatible with RetrievalQA
retriever = vector_store.as_retriever()

100%|██████████| 8/8 [00:01<00:00,  4.44it/s]


In [57]:

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # Other options: "map_reduce", "refine"
    retriever=retriever
)

In [60]:
from IPython.display import display, Markdown

In [61]:
query = "If we accidentally accept Terms of Service without reading, what should we do?"
response = qa_chain.run(query)

display(Markdown(response))

Okay, I understand. Let's analyze the new context and see if it warrants a refinement of the original answer.

**Original Question:** If we accidentally accept Terms of Service without reading, what should we do?

**Original Answer:** A. Review the agreement immediately and adjust settings

**New Context:** The new context consists of three multiple-choice questions related to Terms of Service, focusing on:

*   **Question 3:** "Royalty-free" rights (meaning the service provider can use your content without paying you).
*   **Question 4:** What to check in Terms of Service (specifically, the data use policy).
*   **Question 5:** "Sub-licensable" data (meaning the service provider can license your data to third parties).

**Analysis:**

The new context *does* provide valuable reinforcement and a slightly more nuanced understanding of *why* reviewing the Terms of Service is crucial after accidentally accepting them. The new questions highlight:

1.  **Financial Implications:** "Royalty-free" clauses can mean you lose control over how your content is used and monetized.
2.  **Data Privacy Risks:** The questions emphasize the importance of scrutinizing data use policies (how your data is collected, used, and potentially shared).
3.  **Third-Party Sharing:** "Sub-licensable" clauses reveal that your data may be shared with other companies, a crucial privacy concern.

**Refined Answer:**

Given the context, we can refine the original answer to be more specific and emphasize the key risks:

**Refined Answer:** **A. Review the agreement immediately, paying close attention to sections about data use, royalty-free rights, and sub-licensing, and adjust your settings accordingly.**

**Explanation of Refinement:**

*   **"paying close attention to sections about data use, royalty-free rights, and sub-licensing"**: This addition directly addresses the key themes introduced in the new context. It guides the user on what to look for when reviewing the agreement.
*   **"and adjust your settings accordingly"**: This part of the original answer is still relevant and necessary.

**Why the Refinement is Better:**

The refined answer is superior because it:

*   **Adds Specificity:** It tells the user *what* to look for in the Terms of Service, rather than just saying "review the agreement."
*   **Highlights Key Risks:** It emphasizes the potential consequences of not reading the Terms of Service, specifically concerning data privacy and content ownership.
*   **Maintains Actionability:** It still includes the actionable step of adjusting settings, which is essential for mitigation.

**Conclusion:**

The new context provides strong justification for refining the original answer. The refined answer is more informative and better guides users on how to handle the situation of accidentally accepting Terms of Service without reading them, with a specific focus on understanding the risks highlighted in the provided questions.
